In [1]:
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math, torch, matplotlib as mpl
import matplotlib.pyplot as plt
from torch import tensor

In [2]:
path=Path('/Users/lingling26/.fastai/data/mnist.pkl')

In [3]:
def get_data():
    path=Path('/Users/lingling26/.fastai/data/mnist.pkl')
    with open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

In [4]:
x_train, y_train, x_valid, y_valid = get_data()

In [5]:
train_mean = x_train.mean()

In [6]:
train_std = x_train.std()

In [7]:
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [8]:
def normalize(x, m, s):  return (x-m)/s

In [9]:
normalize(x_train[0], train_mean, train_std)

tensor([-0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
        -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
        -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
        -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
        -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
        -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
        -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
        -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
        -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
        -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
        -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
        -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244, -0.4244,
        -0.4244, -0.4244, -0.4244, -0.42

In [10]:
#通常需要标准化
x_train = normalize(x_train, train_mean, train_std)
#对应的， valid set也需要标准化，但是注意，mean和std需要用train set的值
x_valid = normalize(x_valid, train_mean, train_std)

In [11]:
x_train.mean()

tensor(0.0001)

In [12]:
x_train.std()

tensor(1.)

注意到没有，标准化之后，均值接近0，标准差接近1

In [13]:
n,m = x_train.shape

In [14]:
c=y_train.max()+1

In [15]:
n,m,c

(50000, 784, tensor(10))

# 开始训练一个mlp模型

In [16]:
nh=50

In [17]:
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)

torch.randn()按标准正态分布随机初始化tensor, 一般来说w1是个m*nh的矩阵，后面除以的那个math.sqrt(m) 是个初始化方法，叫做kaiming init或者he init；它的作用是使得第一层隐层输出的结果接近标准正态分布, 这只是一种简便的，近似算法；mean不一定是0，多运行几次这个w1, b1初始化和lin()函数，看下t.mean(), t.std(); 每次多值都不太一样，但是mean基本上是比较小的，std基本上是接近1的

In [18]:
w1

tensor([[-0.0676,  0.0334,  0.0612,  ..., -0.0302,  0.0078, -0.0541],
        [-0.0411, -0.0142,  0.0173,  ...,  0.0462, -0.0454,  0.0228],
        [ 0.0002,  0.0801, -0.0266,  ...,  0.0689,  0.0006,  0.0136],
        ...,
        [-0.0500,  0.0359, -0.0188,  ..., -0.0617, -0.0249,  0.0336],
        [-0.0318, -0.0076,  0.0488,  ...,  0.0034,  0.0089,  0.0269],
        [-0.0057,  0.0205, -0.0240,  ...,  0.0737, -0.0306, -0.0255]])

In [19]:
def lin(x, w, b): return x@w + b

In [20]:
t = lin(x_valid, w1, b1)

In [21]:
t.mean(),t.std()

(tensor(0.1128), tensor(1.0171))

用pytorch内置函数实现relu     
clamp_min(x)内置函数当值小于参数x时，返回x，反之返回值本身

In [22]:
def relu(x): return x.clamp_min(0.)

In [23]:
t1 = relu(lin(x_valid,w1, b1))

In [24]:
t1.mean(),t1.std()

(tensor(0.4602), tensor(0.6356))

发现relu之后，不是标准正态了，因此还是得调节一下
kaiming he提出的方法还是刚刚调节w1的方法，不过这次不是除以math.sqrt(m), 而是math.sqrt(2/m)          
乘以了一个2

In [25]:
w1 = torch.randn(m, nh)*math.sqrt(2/m)

In [26]:
t1 = relu(lin(x_valid,w1,b1))

In [27]:
t1.mean(), t1.std()

(tensor(0.4800), tensor(0.7591))

这样发现std接近1了，但是mean大于0.5了，所以调整一下relu函数

In [28]:
def relu(x): return x.clamp_min(0.) - 0.5

In [29]:
t2 = relu(lin(x_valid, w1, b1))

In [30]:
t2.mean(),t2.std()

(tensor(-0.0200), tensor(0.7591))

现在就接近0,1了

In [31]:
w2 = torch.randn(nh, 1) * math.sqrt(2/nh)
b2 = torch.zeros(1)

In [32]:

def model(xb):
    l1 = relu(lin(xb, w1, b1))
    l2 = lin(l1, w2, b2)
    return l2

In [33]:
%timeit -n 10 _=model(x_valid)

11.7 ms ± 930 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
model(x_valid[0])

tensor([-0.0329])

In [35]:
model(x_valid).shape

torch.Size([10000, 1])

# loss function

squeeze()函数把tensor中的大小为1的维度去掉，如果有多个大小为1的维度，都会被去掉，所以最好指定一下具体哪个维度

In [36]:
def mse(y, target):
    return (y.squeeze(-1) - target).pow(2).mean()

In [37]:
y_valid.shape
model(x_valid).shape
#model(x_valid).squeeze(-1).shape

torch.Size([10000, 1])

In [38]:
mse(model(x_valid), y_valid)

tensor(32.5076)

# back propagation

In [39]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

mse()函数的求导，mse函数的输入shape是[10000,1]；2*（y-target）维度是1， 所以要unsqueeze(-1)变成[10000,1]; 然后还要除以input.shape[0]; 因为mse的时候求了和，所以求导之后要除, 等于求了个均值

tensor.t(), 把tensor转制

In [40]:
model(x_valid).shape[0]
#w1.t()

10000

relu函数的导数是(inp>0).float(), 后面的out.g是chain rule, 把导数串联起来       
inp是前一层的输出（就是本层的输入），back propagation的时候，把导数存在inp.g中。 

In [41]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

一个矩阵的导数，就是这个矩阵的转置     
但是对于linear的导数，我们不仅需要输出层对输入层的导数，还需要对w, 对b的导数，就是说对于linear层，一共需要3个导数。  
w.g是对w求导，所以等于输入x，然后乘以out.g（这样才能链式法则，把后面的导数传递起来），out.g.unsqueeze(1)，为啥是1呢，因为这是这一层矩阵乘法消失的那个维度，即，input的列，output的行，所以反向传播的时候，要把这列展开;      
input unsqueeze的为啥是-1呢，因为linear这一个module， 矩阵乘法，input贡献行，input的列会和output的行消失掉，input无法影响的是output的列，所以这一维需要unsqueeze补上

In [42]:
def lin_grad(inp, out, w, b):
    inp.g = out.g@w.t()
    w.g = (inp.unsqueeze(-1)*out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [43]:
l1 = lin(x_valid, w1, b1)
relu1 = relu(l1)
l2 = lin(relu1, w2, b2)

In [44]:
mse_grad(l2, y_valid)
l2.g

tensor([[-0.0006],
        [-0.0017],
        [-0.0012],
        ...,
        [-0.0009],
        [-0.0014],
        [-0.0016]])

back propagation理解上的几个要点。   
1. 每一层的g，一定和这层原来的shape一样。   
2. lin_grad函数里面，inp.shape是[10000,50], out.g的shape和out一样，是[10000,1], 链式法则要能乘起来，shape必须匹配，所以通过unsqueeze调整shape，记得broadcast规则么？把长度为1的维度自动扩充为可以匹配的维度，所以inp.unsqueeze(-1)变成[10000,50,1], out.g.unsqueeze(1)变成[10000,1,1],  (inp.unsqueeze(-1)*out.g.unsqueeze(1)) 之后[10000,50,1], 再对第一个维度10000求合，变成[50,1]       

In [45]:
lin_grad(relu1, l2, w2, b2)
relu1.unsqueeze(-1).shape
w2.g.shape

torch.Size([50, 1])

In [46]:
relu_grad(l1, relu1)
l1.g
w1.shape

torch.Size([784, 50])

In [47]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [48]:
forward_and_backward(x_valid, y_valid)

把我们一轮求导的结果保存起来

In [50]:
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_valid.g.clone()

下面用pytorch的方法实现上面的功能（一轮正向+反向传播），看看和我们的结果是否一致。一致说明我们的实现没有问题

In [51]:
xt2 = x_valid.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [52]:
def forward(inp, targ):
    l1=inp@w12+b12
    relu1=relu(l1)
    l2=relu1@w22+b22
    
    return mse(l2, targ)

requires_grad_(true)方法是pytorch提供的方法，凡是需要求导的变量（forward函数里面），都需要设置requires_grad, 这样tensor才会生气grad存放的空间；不实现调用这个方法会报错；     
这个fcc模型，需要求导的有5个变量，x_train，w1, b1, w2, b2

In [53]:
loss=forward(xt2, y_valid)

In [54]:
loss.backward()

# 尝试自己实现一个pytorch api

In [55]:
class Mse():
    def __call__(self, inp, targ):
        self.inp=inp
        self.targ=targ
        self.out=(self.inp.squeeze()-self.targ).pow(2).mean(0)
        return self.out
        
    
    def backward(self):
        self.inp.g = 2*(self.inp.squeeze()-self.targ).unsqueeze(-1)/self.inp.shape[0]

python类实现$__init__()$方法，类似c++构造函数;     
实现$__call__()$方法，则可以像调用函数一样使用类的实例，即假设x是类的实例，则x()相当于x.$__call__()$       
实现一个fc, 需要实现 mse, linear, relu 3个module， 每个module再根据具体情况，实现call(相当于是forward)和backward函数

In [56]:
class Relu():
    def __call__(self, inp):
        self.inp=inp
        self.out=self.inp.clamp_min(0.) - 0.5
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g * (self.inp>0).float()

In [57]:
class Linear():
    def __init__(self, w, b): self.w, self.b = w, b
    
    def __call__(self, inp):
        self.inp=inp
        self.out = inp@self.w+self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g@self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [58]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Linear(w1, b1), Relu(), Linear(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, target):
        for l in self.layers: x = l(x)
        return self.loss(x, target)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

上面这个model类很好的诠释了init函数和call函数的使用场景；self.layers变量依次调用各个modules的init函数，init函数返回每个module的对象，放在layers变量中，call函数里，依次遍历layer，然后依次调用module的call函数；(call函数就是直接拿对象名加括号调用)，backward函数使用reversed方法，把layers这个list反转，然后依次调用backward方法

In [59]:
w1.g, b1.g, w2.g, b2.g = [None] * 4
model = Model(w1,b2,w2,b2)

置空各个参数的导数

In [60]:
%time loss = model(x_valid, y_valid)

CPU times: user 25.3 ms, sys: 2.6 ms, total: 27.9 ms
Wall time: 14.2 ms


In [61]:
%time grad = model.backward()

CPU times: user 991 ms, sys: 522 ms, total: 1.51 s
Wall time: 1.31 s


上面的写法很繁琐，下面进一步简化写法，把Module类抽象出来

In [95]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*self.args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
        

所有module其实就实现2个功能，一个forward，一个backward，上面的module类call函数实现初始化和forward，forward，backward本身只是个接口; call函数抽象了每个module被调用的时候的处理逻辑，就是先初始化参数，然后调用forward，后面relu, linear都继承了module类，所以也继承了call函数；然后relu和linear分别重载了forward和backward函数。由于module类中backward接口实际上是调用了bwd函数，所以后面子类里实现的也是bwd函数

In [96]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.) - 0.5
    def bwd(self, out, inp): inp.g = out.g * (inp>0.).float()

In [104]:
class Linear(Module):
    def __init__(self, w, b): self.w,self.b = w,b
    def forward(self, inp): return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = out.g@self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [105]:
class Mse(Module):
    def forward(self, inp, targ): return(inp.squeeze()-targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1)/targ.shape[0]

In [106]:
class Model():
    def __init__(self):
        self.layers = [Linear(w1,b1), Relu(), Linear(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [107]:
w1.g, b1.g, w2.g, b2.g = [None] * 4

In [108]:
model = Model()

In [109]:
%time loss = model(x_valid, y_valid)

CPU times: user 26.1 ms, sys: 2.25 ms, total: 28.4 ms
Wall time: 14 ms


In [110]:
%time model.backward()

CPU times: user 57.3 ms, sys: 8.25 ms, total: 65.5 ms
Wall time: 36 ms
